# Laboratoria 4

In [1]:
from numpy import mean
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
# create dataset
X, y = make_classification(n_samples=1000, n_features=20, random_state=1, n_informative=10, n_redundant=10)

## Fixed folds crossvalidation

In [5]:
?GridSearchCV

In [2]:
## TODO


## Nested resampling

In [2]:
## TODO

## Pipelines

In [ ]:
## TODO